In [74]:
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm
import numpy as np
import random

In [2]:
df = pd.read_csv('imdb_dataset.tsv', sep='\t', header=None)

In [3]:
edges = df.to_records(index=False)

In [4]:
#https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html
# I think the simple regex is sufficient
df[2] = df[1].str.extract(r'(\d{4})', expand=True).fillna(10000).astype(int) #FIXME fillna is pretty ugly rn
df = df.rename(columns={0: "actor", 1: "movie", 2: "year"})

In [5]:
print(df[df['year'].isnull()])

Empty DataFrame
Columns: [actor, movie, year]
Index: []


In [6]:
df[1450:1455]

,actor,movie,year
1450,"Aamschot, Michael",Honeyz (2007),2007
1451,"Aamund, Asger",Hj?lp krigens ofre (2003) (TV),2003
1452,"Aamundson, John",ODC,10000
1453,Aanaahad,Lahore (2010),2010
1454,"Aanderaa, Torgny Gerhard",Citizen X (2007),2007


In [7]:
actors = df.actor.unique()
movies = df.movie.unique()
print(f"Number of actors is: {actors.size} \nNumber of movies is: {movies.size} \nTotal nodes will be: {actors.size + movies.size}")
print(f"Number of edges is will be: {len(edges)}")

Number of actors is: 2364796 
Number of movies is: 745941 
Total nodes will be: 3110737
Number of edges is will be: 8104335


In [8]:
movies_dict = df.drop(columns='actor').drop_duplicates().set_index('movie').to_dict('index')
movies_tuples_list = [(k, v) for k, v in movies_dict.items()] #ugly but convenient for what networkx expects

In [9]:
movies_tuples_list

[('Nykytaiteen museo (1986)', {'year': 1986}),
 ('Suuri illusioni (1985)', {'year': 1985}),
 ('E.R. Sluts (2003) (V)', {'year': 2003}),
 ('American Pimp (1999)', {'year': 1999}),
 ('Beats, Rhymes & Life: The Travels of a Tribe Called Quest (2011)',
  {'year': 2011}),
 ('Gangsta Rap: The Glockumentary (2007)', {'year': 2007}),
 ('Get It Where You Fit in 1 (2003) (V)', {'year': 2003}),
 ('Ghetto Physics (2010)', {'year': 2010}),
 ('Ghostride the Whip (2008) (V)', {'year': 2008}),
 ('Hip Hop Uncensored Vol. 4: Miami Vice (2002) (V)', {'year': 2002}),
 ('Menace II Society (1993)', {'year': 1993}),
 ('Ozone West 3 (2009) (V)', {'year': 2009}),
 ('Pimpalation: Return of the Trill (2006)', {'year': 2006}),
 ('Planet Rock: The Story of Hip-Hop and the Crack Generation (2011) (TV)',
  {'year': 2011}),
 ('Porndogs: The Adventures of Sadie (2009)', {'year': 2009}),
 ('Rhyme & Reason (1997)', {'year': 1997}),
 ('Scarface: Greatest Hits on DVD (2003) (V)', {'year': 2003}),
 ('Stop Pepper Palmer (20

In [10]:
oriGinal = nx.Graph()
oriGinal.add_nodes_from(actors, bipartite = 0) #attribute bipartite following documentation recommendations. In this case 0 is actors, 1 is movies
print(f"Number of nodes after adding actors is {oriGinal.number_of_nodes()}")
oriGinal.add_nodes_from(movies_tuples_list, bipartite = 1)
print(f"Number of nodes after adding movies is {oriGinal.number_of_nodes()}") #???????
      

Number of nodes after adding actors is 2364796
Number of nodes after adding movies is 3110735


In [11]:
oriGinal.add_edges_from(edges)

In [16]:
G = nx.convert_node_labels_to_integers(oriGinal, label_attribute='original_name')

In [17]:
actor_nodes = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0}
movies_nodes = set(G) - actor_nodes

In [18]:
#TODO asserts
print(f"Number of actor nodes: {len(actor_nodes)}")
print(f"Number of movies nodes: {len(movies_nodes)}")
print(f"Total number of nodes: {len(actor_nodes) + len(movies_nodes)}")

print(f"#Nodes? {oriGinal.number_of_nodes() == G.number_of_nodes()}")
print(f"#Edges? {oriGinal.number_of_edges() == G.number_of_edges()}")

Number of actor nodes: 2364794
Number of movies nodes: 745941
Total number of nodes: 3110735
#Nodes? True
#Edges? True


In [20]:
print(oriGinal["'t Hoen, Dani?l"])
print(oriGinal["'Kid Niagara' Kallet, Harry"])

print(G[2151046])

nodes_data = G.nodes.data(True)
print(nodes_data[2365734])

{'Zonde (2010)': {}}
{'Drug Demon Romance (2012)': {}}
{25695: {}, 207265: {}, 457200: {}, 472789: {}, 1009276: {}, 1140040: {}, 1486377: {}, 1751991: {}, 2036767: {}, 2816090: {}, 2313367: {}}
{'bipartite': 1, 'year': 2011, 'original_name': 'To Meet It with Awe (2011)'}


In [21]:
original_data = oriGinal.nodes.data(True)
print(original_data['To Meet It with Awe (2011)'])

{'bipartite': 1, 'year': 2011}


In [22]:
G.size

<bound method Graph.size of <networkx.classes.graph.Graph object at 0x103587880>>

In [21]:
degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
dmax = max(degree_sequence)

## Question 1
G) Considering only the movies up to year x with x in {1930,1940,1950,1960,1970,1980,1990,2000,2010,2020}, write a function which, given x, computes the average number of movies per actor up to year x. 

In [23]:
def avgMoviesPerActorUpToYear(graph, act_nodes, mv_nodes, year):
    movies_up_to_year = {x for x,y in graph.nodes(data=True) if y['bipartite'] == 1 and y['year'] <= year}
    nodes_subset = movies_up_to_year.union(act_nodes) 
    # We have two ways of interpreting the question. One is to consider actors even when they've zero movies, the 
    # other is to consider actors only when they have a non zero counter. Regardless, this is considered later
    subgraph = graph.subgraph(nodes_subset)
    assert subgraph.number_of_nodes() == len(nodes_subset)
    
    subgraph_actor_nodes = {n for n, d in subgraph.nodes(data=True) if d["bipartite"] == 0} #in this case it's not necessary because actors are first nodes (in order), but what I said is not a given
    
    degrees = subgraph.degree(nbunch = subgraph_actor_nodes)
    deg_data = pd.DataFrame(degrees)
    sol = (year, deg_data[1].mean(), deg_data[1].replace(0, np.NaN).mean()) #convenient for output later
    #print(f"Mean: {sol[1]}")
    #print(f"Mean removing zeros: {sol[2]}")
    return sol
    
    #print(a)

In [24]:
for year in range(1930, 2021, 10):
    print(avgMoviesPerActorUpToYear(G, actor_nodes, movies_nodes, year))


(1930, 0.16629989758093094, 6.18496791645697)
(1940, 0.303672539764563, 6.929682524365531)
(1950, 0.4303372725066116, 7.179201557660969)
(1960, 0.5872376198518772, 6.7980360193656715)
(1970, 0.786403382281924, 6.217696718433943)
(1980, 1.0331229696963034, 5.560324089352163)
(1990, 1.3526522817632318, 4.893006066650554)
(2000, 1.816060510978969, 4.343798170482544)
(2010, 2.9111114963924978, 3.623355084828727)
(2020, 3.4247304416367768, 3.426930244288014)


## Question 2
3) Considering only the movies up to year x with x in {1930,1940,1950,1960,1970,1980,1990,2000,2010,2020} and restricting to the largest connected component of the graph. Approximate the closeness centrality for each node. Who are the top-10 actors?

[Fast Approximation of Centrality (D. Eppstein, J. Wang)](https://www.ics.uci.edu/~eppstein/pubs/EppWan-SODA-01.pdf)
```
*Pseudocode
1. Let k be the number of iterations needed to obtain the desired error bound
2. In iteration i, pick vertex v_i uniformly at random from G and solve the SSSP problem with v_i as a source. 
3. Let (1) be the centrality estimator for vertex u
```
Where  
(1)  $\hat{c}_u = \frac{1}{\sum_{i=1}^k \frac{n*d(v_i, u)}{k(n-1)}}$

In [146]:
def closenessCentralityUpToYear(graph, act_nodes, year, k):
    movies_up_to_year = {x for x,y in graph.nodes(data=True) if y['bipartite'] == 1 and y['year'] <= year}
    nodes_subset = movies_up_to_year.union(act_nodes) 
    cc_subgraph = graph.subgraph(nodes_subset)
    largest_cc = max(nx.connected_components(cc_subgraph), key=len) # get largest CC 
    cc_subgraph = graph.subgraph(largest_cc)
    assert cc_subgraph.number_of_nodes() == len(largest_cc)
    #nx.single_source_shortest_path_length(cc_subgraph, 215) #TODO I do this for now, then perhaps (?) with BFS 
    # 2. sample k nodes 
    starting_nodes = random.sample(list(largest_cc), k)
    print(starting_nodes)
    sssp_s = list(map(lambda x: nx.single_source_shortest_path_length(cc_subgraph, x), starting_nodes)) # call single_source_shourtest_path_length(cc, sample) for each sample in samples
    '''
    #I wanted to do a dictionary but we actually don't really care about who generates a sample 
    sssp_s_dict = {}
    for starting_node in tqdm(starting_nodes): 
        sssp_s_dict[starting_node] = nx.single_source_shortest_path_length(cc_subgraph, starting_node)'''
    
    n = len(largest_cc)
    distances_df = pd.DataFrame(sssp).T
    distances_df['centrality'] = distances_df.mean(numeric_only=True, axis=1).apply(lambda x: 1/(x*(n/(n-1)))) #TODO check correctness
    
    
    return distances_df


In [147]:
centralities = closenessCentralityUpToYear(G, actor_nodes, 1970, 3)


[2530605, 319492, 2390015]


In [148]:
centralities

,0,1,2,centrality
2726705,0,6,7,0.230769
232585,1,5,6,0.249999
809616,1,5,6,0.249999
837362,1,5,6,0.249999
1633215,1,7,6,0.214285
...,...,...,...,...
652618,17,17,16,0.060000
1054028,17,17,16,0.060000
1049459,17,17,16,0.060000
143486,17,17,14,0.062500


In [142]:
#distances_df = pd.DataFrame(sssp).T
#n = 5
#distances_df['centrality'] = distances_df.mean(numeric_only=True, axis=1).apply(lambda x: 1/(x*(n/(n-1))))
                                                                         

In [143]:
distances_df

,0,1,2,avg,centrality
1130121,0,5,8,0.174545,0.242893
2421064,1,4,7,0.188235,0.262548
1442048,2,3,6,0.204255,0.285606
249220,2,5,8,0.152381,0.211188
885125,2,5,8,0.152381,0.211188
...,...,...,...,...,...
652618,16,17,18,0.046377,0.062688
1054028,16,17,18,0.046377,0.062688
1049459,16,17,18,0.046377,0.062688
143486,16,17,18,0.046377,0.062688


## Question 3
III) Which is the pair of movies that share the largest number of actors?

## Question 4
Build also the actor graph, whose nodes are only actors and two actors are connected if they did a movie together. Answer to the following question:

Which is the pair of actors who collaborated the most among themselves?

### Notes
- [NetworkX docs on bipartite graphs](https://networkx.org/documentation/stable/reference/algorithms/bipartite.html) However, if the input graph is not connected, there are more than one possible colorations. This is the reason why we require the user to pass a container with all nodes of one bipartite node set as an argument to most bipartite functions.
- Networkx uses a dictionary of dictionaries of dictionaries, as specified in the docs. NetworkX uses a “dictionary of dictionaries of dictionaries” as the basic network data structure. This allows fast lookup with reasonable storage for large sparse networks. The keys are nodes so G[u] returns an adjacency dictionary keyed by neighbor to the edge attribute dictionary. A view of the adjacency data structure is provided by the dict-like object G.adj as e.g. for node, nbrsdict in G.adj.items():. The expression G[u][v] returns the edge attribute dictionary itself. A dictionary of lists would have also been possible, but not allow fast edge detection nor convenient storage of edge data.